In [1]:
# Load the modules we need
import os
from bs4 import BeautifulSoup
import json
import re
from datetime import datetime

In [2]:
% pwd

'/Users/john/Code/tedmetadata'

In [13]:
# FILE being examined
the_data = "hand/david_binder_the_arts_festival_revolution.html"

import glob

file_list = glob.glob('./hand' + '/*.html')

for filename in file_list:
    with open(filename, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), "html5lib")
        my_list = [i.string.lstrip(
            'q("talkPage.init", {\n\t"el": "[data-talk-page]",\n\t "__INITIAL_DATA__":')
           .rstrip('})')
           for i in soup.select('script') 
           if i.string and i.string.startswith('q')]
        # Read first layer of JSON and get out those elements we want
        pre_json = '{"' + "".join(my_list)
        my_json = json.loads(pre_json)
        slug = my_json['slug']
        vcount = my_json['viewed_count']
        event = my_json['event']
        # Read second layer of JSON and get out listed elements:
        properties = "filmed,published" # No spaces between terms!
        talks_listed = str(my_json['talks']).split(",")
        regex_list = [".*("+i+").*" for i in properties.split(",")]
        matches = []
        for e in regex_list:
            filtered = filter(re.compile(e).match, talks_listed)
            indexed = "".join(filtered).split(":")[1]
            print(indexed)

            date = re.search('[0-9]{10}', indexed).group(0)
            matches.append(date)
        filmed = datetime.utcfromtimestamp(float(matches[0])).strftime('%Y-%m-%d')
        #published = datetime.utcfromtimestamp(float(matches[1])).strftime('%Y-%m-%d')
        print("{},{},{},{}".format(slug, vcount, event, filmed))

//dsc.discovery.com/show-news/top-10-startling-giant-squid-facts.html" target="_blank">filmed the giant squid</a>  (<em>Architeuthis</em>) for the first time in its home ocean.</p>' 'filmed'


AttributeError: 'NoneType' object has no attribute 'group'

ALTERNATE: if we change single quotes to double quotes: will the `json` module accept it? The short answer is that the `json` module chokes on a variety of things, including the Arabic and Chinese characters inside quotation marks. 

```python
json_string = str(my_json['talks']).replace("'", '"').replace('None', '"None"').replace('True', '"True"')
# print(json_string)
nested_json = json.loads(json_string)
```

But while I was working on this, I came across a better way to replace multiple items than my chain above:

```python
replacements = {'None': '"None"', 'True': '"True"'} # define desired replacements here

rep = dict((re.escape(k), v) for k, v in replacments.items())
pattern = re.compile("|".join(rep.keys()))
text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
```

In [ ]:
print(slug, vcount, event, filmed, published)